In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as s
import pickle

import warnings
warnings.filterwarnings('ignore')

import datetime as dt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from scipy.stats import shapiro

import AlgoData as alg

In [98]:
df = alg.ReadTrainData('train_final.csv', raw=False)
df.drop('Unnamed: 0', axis=1, inplace=True)

(18707, 18)
['Manufacturer', 'Model', 'Category', 'Fuel type', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel', 'Color']


In [99]:
df.columns

Index(['index', 'Price', 'Levy', 'Prod. year', 'Leather interior',
       'Engine volume', 'Mileage', 'Cylinders', 'Airbags', 'Turbo',
       'Mileage_BIN', 'EngineVolume_BIN', 'Manufacturer', 'Model', 'Category',
       'Fuel type', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel',
       'Color'],
      dtype='object')

In [100]:
X = df.drop(['Price', 'index'], axis=1)
y = df['Price']
X.head()


,Levy,Prod. year,Leather interior,Engine volume,Mileage,Cylinders,Airbags,Turbo,Mileage_BIN,EngineVolume_BIN,Manufacturer,Model,Category,Fuel type,Gear box type,Drive wheels,Doors,Wheel,Color
0,1399.0,2010,1,3.41,186005.0,6.0,12,0,5.0,4.0,21.0,1180.0,4.0,2.0,0.0,0.0,1.0,0.0,12.0
1,1018.0,2011,0,3.00,192000.0,6.0,8,0,5.0,4.0,5.0,632.0,4.0,5.0,2.0,0.0,1.0,0.0,1.0
2,0.0,2006,0,1.30,200000.0,4.0,2,0,5.0,0.0,14.0,656.0,3.0,5.0,3.0,1.0,1.0,1.0,1.0
3,862.0,2011,1,2.50,168966.0,4.0,0,0,4.0,3.0,11.0,635.0,4.0,2.0,0.0,0.0,1.0,0.0,14.0
4,446.0,2014,1,1.30,91901.0,4.0,4,0,2.0,0.0,14.0,656.0,3.0,5.0,0.0,1.0,1.0,0.0,12.0


In [101]:
#Randomized Search CV for searching the best parameters

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200], 'max_features': ['auto', 'sqrt'], 'max_depth': [5, 10, 15, 20, 25, 30], 'min_samples_split': [2, 5, 10, 15, 100], 'min_samples_leaf': [1, 2, 5, 10]}


In [102]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestRegressor()

In [103]:
rf_random_model = RandomizedSearchCV(estimator = rf,
                                     param_distributions = random_grid,
                                    scoring='neg_mean_squared_error',
                                    n_iter = 10,
                                    cv = 5,
                                    verbose=2,
                                    random_state=42,
                                    n_jobs = 1)

In [116]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=10)
Xtrain.head()

,Levy,Prod. year,Leather interior,Engine volume,Mileage,Cylinders,Airbags,Turbo,Mileage_BIN,EngineVolume_BIN,Manufacturer,Model,Category,Fuel type,Gear box type,Drive wheels,Doors,Wheel,Color
8540,1268.0,2012,0,1.50,90000.0,4.0,4,0,2.0,1.0,14.0,706.0,7.0,5.0,0.0,1.0,1.0,1.0,12.0
11487,1360.0,2013,1,3.41,214763.0,8.0,12,0,6.0,4.0,24.0,747.0,4.0,5.0,0.0,0.0,1.0,0.0,1.0
14002,585.0,2013,1,1.80,156326.0,4.0,12,0,4.0,1.0,15.0,607.0,9.0,5.0,0.0,1.0,1.0,0.0,7.0
18521,0.0,1996,0,1.80,11111.0,6.0,16,0,0.0,1.0,11.0,992.0,3.0,5.0,1.0,1.0,1.0,0.0,14.0
18544,0.0,2003,0,1.70,253033.0,4.0,1,1,7.0,1.0,28.0,468.0,2.0,1.0,1.0,1.0,0.0,0.0,1.0


In [72]:
Xtest = Xtest.drop('index', axis=1)


KeyError: "['index'] not found in axis"

In [105]:
y.describe()

count    18707.000000
mean         9.111365
std          1.404353
min          4.605170
25%          8.665958
50%          9.495294
75%         10.007780
max         11.507310
Name: Price, dtype: float64

In [106]:
rf_random_model.fit(Xtrain, ytrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   8.7s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   6.9s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   7.0s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   6.9s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   6.9s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time=  10.7s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time=  10.7s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimator

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=1,
                   param_distributions={'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              100],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100,
                                                         1200]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [107]:
#beta RF final 0.0.5
import joblib

In [109]:
y_pred = rf_random_model.predict(Xtest)

In [110]:
MAPE = mean_absolute_percentage_error(np.exp(ytest), np.exp(y_pred))

In [111]:
MAPE


0.9977118915230091

In [112]:
MAE = mean_absolute_error(np.exp(ytest), np.exp(y_pred))
MAE

4136.277131325615

In [117]:
model = joblib.load('C:\\Users\\91748\\Desktop\\beta_RF_Final_0_0_6.obj')

In [118]:
y_pred = model.predict(Xtest)


In [120]:
print(mean_absolute_error(np.exp(ytest), np.exp(y_pred)))
print(mean_absolute_percentage_error(np.exp(ytest), np.exp(y_pred)))

4136.277131325615
0.9977118915230091


#### Testing encoder


In [121]:
f = open('Encoder.bin', 'rb')
enc = pickle.load(f)
f.close()

In [122]:
X = enc.inverse_transform(X.get(['Manufacturer', 'Model', 'Category', 'Fuel type', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel', 'Color']))

In [123]:
X.shape

(18707, 9)

In [124]:
df_categ = pd.DataFrame(X, columns=['Manufacturer', 'Model', 'Category', 'Fuel type', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel', 'Color'])

In [125]:
df_categ.head()

,Manufacturer,Model,Category,Fuel type,Gear box type,Drive wheels,Doors,Wheel,Color
0,LEXUS,RX 450,Jeep,Hybrid,Automatic,4x4,04-May,Left wheel,Silver
1,CHEVROLET,Equinox,Jeep,Petrol,Tiptronic,4x4,04-May,Left wheel,Black
2,HONDA,FIT,Hatchback,Petrol,Variator,Front,04-May,Right-hand drive,Black
3,FORD,Escape,Jeep,Hybrid,Automatic,4x4,04-May,Left wheel,White
4,HONDA,FIT,Hatchback,Petrol,Automatic,Front,04-May,Left wheel,Silver


In [43]:
df_categ['Model'].value_counts()

Sonata          1066
Prius           1010
Camry            932
Elantra          906
E 350            539
                ... 
Passat sport       1
E 320 4×4          1
Feroza             1
C-MAX C-MAX        1
Prius C aqua       1
Name: Model, Length: 1510, dtype: int64

In [47]:
df_new = alg.ReadTrainData('train_final.csv', raw=True, encode=False)
df_new.drop('Unnamed: 0', axis=1, inplace=True)


(18707, 18)


In [48]:
df_new.Model.value_counts()

Sonata          1066
Prius           1010
Camry            932
Elantra          906
E 350            539
                ... 
Passat sport       1
E 320 4×4          1
Feroza             1
C-MAX C-MAX        1
Prius C aqua       1
Name: Model, Length: 1510, dtype: int64

# Eliminating 'Index' column


In [51]:
df =df.drop('index', axis=1)